#using astroquery to get comet ra,decs in a date range or list of specific dates

In [2]:
#essential python packages
import numpy as np
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons


In [3]:
'''
A good reference for comets is Yan Fernandez's list of comets:

https://physics.ucf.edu/~yfernandez/cometlist.html

There are >800 as of Nov 2023
'''


"\nA good reference for comets is Yan Fernandez's list of comets:\n\nhttps://physics.ucf.edu/~yfernandez/cometlist.html\n\nThere are >800 as of Nov 2023\n"

In [27]:
#JPL astroquery functions

#range of dates

def get_current_pos_date_range(object_name, start_date, end_date, time_step):
    obj = Horizons(id=object_name, epochs={'start':start_date, 'stop':end_date,'step':time_step})
    try:
        eph = obj.ephemerides()
        pos = SkyCoord(eph['RA'], eph['DEC'], unit='deg')
        #times = Time(np.asarray(eph['datetime_jd']), format='jd', scale='utc')
        times = np.asarray(eph['datetime_jd'])
        ra,dec,times = np.asarray(pos.ra),np.asarray(pos.dec),times
    except ValueError:
        print ('Problem with ' + object_name +'. Too many object entries with the same name in JPL database. Need to use record number as object name, e.g., 90000031')
        ra,dec,times = np.asarray('0'),np.asarray('0'),'0'
    return ra,dec,times

def get_current_pos_date_list(object_name, epochs_jd):#need to keep less than ~1000 entries per querie
    obj = Horizons(id=object_name, epochs=epochs_jd)
    try:
        eph = obj.ephemerides()
        pos = SkyCoord(eph['RA'], eph['DEC'], unit='deg')
        #times = Time(np.asarray(eph['datetime_jd']), format='jd', scale='utc')
        times = np.asarray(eph['datetime_jd'])
        ra,dec,times = np.asarray(pos.ra),np.asarray(pos.dec),times
    except ValueError:
        print ('Problem with ' + object_name +'. Too many object entries with the same name in JPL database. Need to use record number as object name, e.g., 90000031')
        ra,dec,times = np.asarray('0'),np.asarray('0'),'0'
    return ra,dec,times

def get_current_pos_date_list_arbitrary_number(object_name, epochs_jd, entry_limit_per_query_int):#want to keep entry_limit_per_query ~300
    if len(epochs_jd)<entry_limit_per_query_int + 1:#do just one batch if number of entries is under 1k.
        obj = Horizons(id=object_name, epochs=epochs_jd)
        try:
            eph = obj.ephemerides()
            pos = SkyCoord(eph['RA'], eph['DEC'], unit='deg')
            #times = Time(np.asarray(eph['datetime_jd']), format='jd', scale='utc')
            times = np.asarray(eph['datetime_jd'])
            ra,dec,times = np.asarray(pos.ra),np.asarray(pos.dec),times
        except ValueError:
            print ('Problem with ' + object_name +'. Too many object entries with the same name in JPL database. Need to use record number as object name, e.g., 90000031, or too many entries are being queried at once.')
            ra,dec,times = np.asarray('0'),np.asarray('0'),'0'
    if len(epochs_jd)>entry_limit_per_query_int:
        obj = Horizons(id=object_name, epochs=epochs_jd[:entry_limit_per_query_int])
        ra,dec,times = np.array([]),np.array([]),np.array([])
        try:
            print ("Computing ephemerides for entries 1 to "+ str(entry_limit_per_query_int) + " entries of " + str(len(epochs_jd)) + " total entries.")
            eph = obj.ephemerides()
            pos = SkyCoord(eph['RA'], eph['DEC'], unit='deg')
            #times_from_JPL = Time(np.asarray(eph['datetime_jd']), format='jd', scale='utc')
            times_from_JPL = np.asarray(eph['datetime_jd'])
            ra,dec,times = np.append(ra,np.asarray(pos.ra)), np.append(dec,np.asarray(pos.dec)), np.append(times,times_from_JPL)
            #now check if first batch is over 1K and do the rest
            fraction_over_interval, number_intervals = np.modf((len(epochs_jd)*1.0)/entry_limit_per_query_int)
            if number_intervals>1.0:
                for interval_tracker in range(1,int(number_intervals)):
                    print ("Computing ephemerides for entries " + str(interval_tracker*entry_limit_per_query_int) +  " to " + str((interval_tracker+1)*entry_limit_per_query_int) + " entries of " + str(len(epochs_jd)) + " total entries.")
                    obj = Horizons(id=object_name, epochs=epochs_jd[interval_tracker*entry_limit_per_query_int:(interval_tracker+1)*entry_limit_per_query_int])
                    eph = obj.ephemerides()
                    pos = SkyCoord(eph['RA'], eph['DEC'], unit='deg')
                    #times_from_JPL = Time(np.asarray(eph['datetime_jd']), format='jd', scale='utc')
                    times_from_JPL = np.asarray(eph['datetime_jd'])
                    ra,dec,times = np.append(ra,np.asarray(pos.ra)), np.append(dec,np.asarray(pos.dec)), np.append(times,times_from_JPL)
            if fraction_over_interval>0.0:#do the batch <1000 over the initial 1k batch
                print ("Computing ephemerides for entries " + str(interval_tracker*entry_limit_per_query_int) +  " to " + str((interval_tracker+1)*entry_limit_per_query_int) + " entries of " + str(len(epochs_jd)) + " total entries.")
                obj = Horizons(id=object_name, epochs=epochs_jd[(interval_tracker+1)*entry_limit_per_query_int:])
                eph = obj.ephemerides()
                pos = SkyCoord(eph['RA'], eph['DEC'], unit='deg')
                #times_from_JPL = Time(np.asarray(eph['datetime_jd']), format='jd', scale='utc')
                times_from_JPL = np.asarray(eph['datetime_jd'])
                ra,dec,times = np.append(ra,np.asarray(pos.ra)), np.append(dec,np.asarray(pos.dec)), np.append(times,times_from_JPL)
        except ValueError:#will catch stuff before the subsequent calls after the the first 1k entries
            print ('Problem with ' + object_name +'. Too many object entries with the same name in JPL database. Need to use record number as object name, e.g., 90000031, or too many entries are being queried at once.')
            ra,dec,times = np.asarray('0'),np.asarray('0'),'0'
        return ra,dec,times

In [28]:
#examples

#for range of dates

object_name, start_date, end_date, time_step = "C/2018 W1", "2023-11-29", "2023-11-30", "10m"

ra_deg, dec_deg, times = get_current_pos_date_range(object_name, start_date, end_date, time_step)

print (ra_deg, dec_deg, times)

(array([245.1596 , 245.16014, 245.16068, 245.16121, 245.16175, 245.16229,
       245.16283, 245.16336, 245.1639 , 245.16444, 245.16498, 245.16551,
       245.16605, 245.16659, 245.16713, 245.16766, 245.1682 , 245.16874,
       245.16928, 245.16982, 245.17035, 245.17089, 245.17143, 245.17197,
       245.1725 , 245.17304, 245.17358, 245.17412, 245.17465, 245.17519,
       245.17573, 245.17627, 245.1768 , 245.17734, 245.17788, 245.17842,
       245.17895, 245.17949, 245.18003, 245.18057, 245.1811 , 245.18164,
       245.18218, 245.18272, 245.18325, 245.18379, 245.18433, 245.18487,
       245.1854 , 245.18594, 245.18648, 245.18702, 245.18755, 245.18809,
       245.18863, 245.18917, 245.1897 , 245.19024, 245.19078, 245.19132,
       245.19185, 245.19239, 245.19293, 245.19347, 245.194  , 245.19454,
       245.19508, 245.19562, 245.19615, 245.19669, 245.19723, 245.19777,
       245.1983 , 245.19884, 245.19938, 245.19992, 245.20045, 245.20099,
       245.20153, 245.20207, 245.2026 , 245.20314,

In [29]:
#queries a list of specific dates


object_name = "342P"

epochs_jd = np.array(['2459846.6283218',
'2459846.6288773',
'2459846.6293403',
'2459846.6298264',
'2459846.6303009',
'2459847.6079167',
'2459847.6083912',
'2459847.6088542',
'2459847.6093287',
'2459847.6098032',
'2459847.6103241',
'2459847.6107986',
'2459847.6112731',
'2459847.6117361',
'2459847.6122106',
'2459847.6127315',
'2459847.613206'])

ra_deg, dec_deg, times = get_current_pos_date_list(object_name, epochs_jd)
print (ra_deg, dec_deg, times)

(array([258.99699, 258.99704, 258.99709, 258.99714, 258.99718, 259.09494,
       259.09498, 259.09503, 259.09508, 259.09513, 259.09518, 259.09523,
       259.09528, 259.09532, 259.09537, 259.09543, 259.09547]), array([-34.02929, -34.02927, -34.02925, -34.02924, -34.02922, -33.99705,
       -33.99703, -33.99702, -33.997  , -33.99699, -33.99697, -33.99695,
       -33.99694, -33.99692, -33.99691, -33.99689, -33.99687]), array([2459846.62832181, 2459846.6288773 , 2459846.6293403 ,
       2459846.6298264 , 2459846.6303009 , 2459847.6079167 ,
       2459847.6083912 , 2459847.6088542 , 2459847.6093287 ,
       2459847.6098032 , 2459847.6103241 , 2459847.6107986 ,
       2459847.6112731 , 2459847.6117361 , 2459847.6122106 ,
       2459847.6127315 , 2459847.613206  ]))


In [30]:
#error example

object_name = "1P"

epochs_jd = np.array(['2459846.6283218',
'2459846.6288773',
'2459846.6293403',
'2459846.6298264',
'2459846.6303009',
'2459847.6079167',
'2459847.6083912',
'2459847.6088542',
'2459847.6093287',
'2459847.6098032',
'2459847.6103241',
'2459847.6107986',
'2459847.6112731',
'2459847.6117361',
'2459847.6122106',
'2459847.6127315',
'2459847.613206'])

ra_deg, dec_deg, times = get_current_pos_date_list(object_name, epochs_jd)
print (ra_deg, dec_deg, times)

Problem with 1P. Too many object entries with the same name in JPL database. Need to use record number as object name, e.g., 90000031
(array('0', dtype='|S1'), array('0', dtype='|S1'), '0')


In [31]:

#break up a large >300 entry query into a few 300 at a time

object_name = "C/2018 W1"
jd_start = 2459846.0
jd_end = 2459881.0
number_entries = 1334
entry_limit_per_query_int = 100
epochs_jd = np.linspace(jd_start, jd_end, number_entries)

ra_deg, dec_deg, times = get_current_pos_date_list_arbitrary_number(object_name, epochs_jd, entry_limit_per_query_int)

print (ra_deg, dec_deg, times)

Computing ephemerides for entries 1 to 100 entries of 1334 total entries.
Computing ephemerides for entries 100 to 200 entries of 1334 total entries.
Computing ephemerides for entries 200 to 300 entries of 1334 total entries.
Computing ephemerides for entries 300 to 400 entries of 1334 total entries.
Computing ephemerides for entries 400 to 500 entries of 1334 total entries.
Computing ephemerides for entries 500 to 600 entries of 1334 total entries.
Computing ephemerides for entries 600 to 700 entries of 1334 total entries.
Computing ephemerides for entries 700 to 800 entries of 1334 total entries.
Computing ephemerides for entries 800 to 900 entries of 1334 total entries.
Computing ephemerides for entries 900 to 1000 entries of 1334 total entries.
Computing ephemerides for entries 1000 to 1100 entries of 1334 total entries.
Computing ephemerides for entries 1100 to 1200 entries of 1334 total entries.
Computing ephemerides for entries 1200 to 1300 entries of 1334 total entries.
Computi